# 📺 Deep Dive: OLED Control (SSD1306)

This notebook is your master guide to controlling the 0.96'' I2C OLED screen with the Raspberry Pi Pico.

## 1. The Physical Language: I2C
The screen and Pico talk using **I2C**. It's a "bus" protocol, meaning you can have multiple devices on the same wires as long as they have different **Addresses**.

### Your Hardware Setup:
- **SDA (Data):** GP16 (Pin 21)
- **SCL (Clock):** GP17 (Pin 22)
- **VCC:** 3.3V (Pin 36)
- **GND:** Ground (Pin 38)
- **I2C Address:** 0x3C (Hexadecimal)

> **Warning:** Do not plug the OLED directly into the Pico headers! The pin order on the OLED (VCC-GND-SCL-SDA) does not match the Pico. You **must** use jumper wires.

## 2. Theoretical Limit: The Grid
The screen is 128 pixels wide and 64 pixels tall. Coordinates start at **(0,0)** in the Top-Left.

```text
(0,0) .-------------------. (127, 0)
      |                   |
      |      128 x 64     |
      |       Pixels      |
      |                   |
(0,63)'-------------------' (127, 63)
```

### Common Coordinates:
- **Center:** (64, 32)
- **Right Edge:** X = 127
- **Bottom Edge:** Y = 63

## 3. The Digital Workflow: FrameBuffer
MicroPython uses a **FrameBuffer** (a block of RAM) to store your drawing before showing it on the glass. 

**The 3-Step Cycle:**
1. **Clear:** `oled.fill(0)` (Removes the old drawing).
2. **Draw:** `oled.text()`, `oled.line()`, etc. (Draws on the RAM, not the screen yet).
3. **Show:** `oled.show()` (Sends the RAM data to the visible screen).

## 4. Method Cheat Sheet

| Command | Description | Example |
| :--- | :--- | :--- |
| `.fill(val)` | Fill screen: 0 (Black), 1 (White) | `oled.fill(0)` |
| `.text(str, x, y)` | Print text at X,Y | `oled.text("Hi", 0, 0)` |
| `.pixel(x, y, val)` | Draw a single dot | `oled.pixel(64, 32, 1)` |
| `.hline(x, y, w, c)` | Draw a Horizontal Line | `oled.hline(0, 10, 128, 1)` |
| `.vline(x, y, h, c)` | Draw a Vertical Line | `oled.vline(10, 0, 64, 1)` |
| `.line(x1, y1, x2, y2, c)` | Draw line between two pts | `oled.line(0,0, 127,63, 1)` |
| `.rect(x, y, w, h, c)` | Draw an empty rectangle | `oled.rect(5, 5, 20, 20, 1)` |
| `.fill_rect(x, y, w, h, c)` | Draw a solid rectangle | `oled.fill_rect(5, 5, 20, 20, 1)` |
| **`.show()`** | **CRITICAL: Updates the screen** | `oled.show()` |

In [ ]:
from machine import Pin, I2C
from ssd1306 import SSD1306_I2C
import time

# 1. Initialize Communication
i2c = I2C(0, sda=Pin(16), scl=Pin(17), freq=400000)
oled = SSD1306_I2C(128, 64, i2c)

# 2. Demo Drawing
def demo():
    oled.fill(0)
    
    # Draw a Border
    oled.rect(0, 0, 128, 64, 1)
    
    # Draw Decorative Lines
    oled.line(0, 0, 127, 63, 1)
    oled.line(0, 63, 127, 0, 1)
    
    # Draw Text in a Clear Box
    oled.fill_rect(30, 22, 68, 20, 0) # Black box to clear lines
    oled.rect(30, 22, 68, 20, 1)      # White outline
    oled.text("SUCCESS", 36, 28, 1)
    
    oled.show()

if __name__ == "__main__":
    demo()

## 5. Troubleshooting
- **`OSError: [Errno 110] ETIMEDOUT`**: The Pico cannot see the screen. Check your wiring and power.
- **No Error, but Screen is Black**: You likely forgot to call `oled.show()` or your wiring is loose.
- **Garbage/Glitchy Image**: Usually means I2C interference. Keep wires short and avoid crossing them over motor power wires.